# Setting up

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style = "whitegrid")

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
# %%time
# path = 'D://Coding//XTX Forecasting Challenge//data-training.csv'
# df = pd.read_csv(path)

In [21]:
%%time
path = 'D://Coding//XTX Forecasting Challenge//data-training.file'
df = pd.read_feather(path, use_threads=8)

Wall time: 5.74 s


In [54]:
df = df.astype('float32')

# Exploratory Data Analysis

In [22]:
# # Plot of [y] distribution
# fig, ax = plt.subplots(figsize=(15,8))
# sns.kdeplot(df.y, bw=0.01)

In [23]:
%%time
# Some feature engineering
df['spread'] = df.askRate0 - df.bidRate0
df['midRate'] = (df.askRate0 + df.bidRate0) / 2
df['expectedY'] = df.midRate.diff(87).shift(-87).clip(-5,5)

Wall time: 293 ms


In [24]:
# # Figuring out what [y] is
# # y(t) is midRate(t+87) - midRate(t), clipped to (-5.5)
# sum(df.y == df.expectedY)
# df.loc[df.y != df.expectedY]

# fig, ax = plt.subplots(figsize=(15,8))
# i = 500
# shift = 87
# # plt.plot(df.index[0:i], df.midRate[0:i].diff(shift).shift(-shift+14))
# plt.plot(df.index[-i:], df.midRate[-i:].diff(shift).shift(-shift))
# plt.plot(df.index[-i:], df.y[-i:])
# plt.legend(('midRate', 'y'))

# Feature engineering

### Fill nans with zeroes

In [83]:
df.fillna(0, inplace=True)

### Basics

In [22]:
bidSizeList = ['bidSize' + str(i) for i in range(0,15)]
askSizeList = ['askSize' + str(i) for i in range(0,15)]
bidRateList = ['bidRate' + str(i) for i in range(0,15)]
askRateList = ['askRate' + str(i) for i in range(0,15)]

In [27]:
%%time
df['totalBidVol1'] = df.bidSize0 + df.bidSize1
for i in range(2,15):
    df['totalBidVol' + str(i)] = df['totalBidVol' + str(i-1)] + df['bidSize' + str(i)]

df['totalAskVol1'] = df.askSize0 + df.askSize1
for i in range(2,15):
    df['totalAskVol' + str(i)] = df['totalAskVol' + str(i-1)] + df['askSize' + str(i)]
    
for i in range(1,15):
    df['bidAskRatio' + str(i)] = df['totalBidVol' + str(i)] / df['totalAskVol' + str(i)]

Wall time: 903 ms


In [200]:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

2498932736


In [94]:
%%time
df['vwaBid1'] = np.einsum('ij,ji->i', df[bidRateList], df[bidSizeList].T) / df[bidSizeList].sum(axis=1)
df['vwaAsk1'] = np.einsum('ij,ji->i', df[askRateList], df[askSizeList].T) / df[askSizeList].sum(axis=1)

Wall time: 2.27 s


In [95]:
df.head()

,askRate0,askRate1,askRate2,askRate3,askRate4,askRate5,askRate6,askRate7,askRate8,askRate9,askRate10,askRate11,askRate12,askRate13,askRate14,askSize0,askSize1,askSize2,askSize3,askSize4,askSize5,askSize6,askSize7,askSize8,askSize9,askSize10,askSize11,askSize12,askSize13,askSize14,bidRate0,bidRate1,bidRate2,bidRate3,bidRate4,bidRate5,bidRate6,bidRate7,bidRate8,bidRate9,bidRate10,bidRate11,bidRate12,bidRate13,bidRate14,bidSize0,bidSize1,bidSize2,bidSize3,bidSize4,bidSize5,bidSize6,bidSize7,bidSize8,bidSize9,bidSize10,bidSize11,bidSize12,bidSize13,bidSize14,y,spread,midRate,expectedY,totalBidVol1,totalBidVol2,totalBidVol3,totalBidVol4,totalBidVol5,totalBidVol6,totalBidVol7,totalBidVol8,totalBidVol9,totalBidVol10,totalBidVol11,totalBidVol12,totalBidVol13,totalBidVol14,totalAskVol1,totalAskVol2,totalAskVol3,totalAskVol4,totalAskVol5,totalAskVol6,totalAskVol7,totalAskVol8,totalAskVol9,totalAskVol10,totalAskVol11,totalAskVol12,totalAskVol13,totalAskVol14,bidAskRatio1,bidAskRatio2,bidAskRatio3,bidAskRatio4,bidAskRatio5,bidAskRatio6,bidAskRatio7,bidAskRatio8,bidAskRatio9,bidAskRatio10,bidAskRatio11,bidAskRatio12,bidAskRatio13,bidAskRatio14,vwaBid,vwaAsk,vwaBidDMid,vwaAskDMid,deltaVBid,deltaVAsk,VOI,vwaBid1,vwaAsk1
0,1620.0,1620.0,1621.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1602.0,1601.0,1600.0,7.0,10.0,1.0,10.0,20.0,3.0,20.0,27.0,11.0,14.0,35.0,10.0,1.0,10.0,13.0,-0.5,4.5,1617.0,-0.5,17.0,18.0,28.0,48.0,51.0,71.0,98.0,109.0,123.0,158.0,168.0,169.0,179.0,192.0,11.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.545898,0.51416,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.793113,11.669204,1604.206887,-1605.330796,0.0,0.0,0.0,1606.135376,1620.685669
1,1620.0,1620.0,1621.0,1622.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,24.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1602.0,1601.0,1600.0,7.0,10.0,1.0,10.0,20.0,3.0,20.0,27.0,11.0,14.0,35.0,10.0,1.0,10.0,13.0,-0.5,4.5,1617.0,-0.5,17.0,18.0,28.0,48.0,51.0,71.0,98.0,109.0,123.0,158.0,168.0,169.0,179.0,192.0,11.0,35.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.545898,0.51416,0.700195,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.793113,10.000617,1604.206887,-1606.999383,0.0,0.0,0.0,1606.135376,1620.849976
2,1620.0,1620.0,1621.0,1622.0,1622.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,24.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1602.0,1601.0,1600.0,7.0,10.0,1.0,10.0,20.0,3.0,20.0,27.0,11.0,14.0,35.0,10.0,1.0,10.0,13.0,-0.5,4.5,1617.0,-0.5,17.0,18.0,28.0,48.0,51.0,71.0,98.0,109.0,123.0,158.0,168.0,169.0,179.0,192.0,11.0,35.0,40.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.545898,0.51416,0.700195,1.142578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.793113,8.399506,1604.206887,-1608.600494,0.0,0.0,0.0,1606.135376,1620.904785
3,1620.0,1620.0,1621.0,1622.0,1622.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,24.0,5.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1602.0,1601.0,1600.0,7.0,10.0,1.0,10.0,20.0,3.0,20.0,27.0,11.0,14.0,35.0,10.0,1.0,10.0,13.0,-0.5,4.5,1617.0,-0.5,17.0,18.0,28.0,48.0,51.0,71.0,98.0,109.0,123.0,158.0,168.0,169.0,179.0,192.0,11.0,35.0,40.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.545898,0.51416,0.700195,0.774414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.793113,12.401974,1604.206887,-1604.598026,0.0,0.0,0.0,1606.135376,1621.258057
4,1620.0,1620.0,1621.0,1622.0,1622.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,24.0,5.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1615.0,1614.0,1613.0,1612.0,1611.0,1610.0,1607.0,1606.0,1605.0,1604.0,1603.0,1602.0,1602.0,1601.0,1600.0,7.0,10.0,1.0,10.0,20.0,3.0

In [42]:
%%time
df['vwaBidDMid'] = df.midRate - df.vwaBid
df['vwaAskDMid'] = df.vwaAsk - df.midRate

Wall time: 127 ms


In [215]:
%%time
df.to_feather('intermediate.file')

Wall time: 3.58 s


In [6]:
%%time
df = pd.read_feather('intermediate.file')

Wall time: 5.75 s


C:\Users\Tym\Anaconda3\envs\xtx\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


# Ideas from papers

In [49]:
%%time
# Volume Order Imbalance
# I still disagree with cancelled orders..
b1, a1 = (df.bidRate0 < df.bidRate0.shift(-1)), (df.askRate0 < df.askRate0.shift(-1))
b2, a2 = (df.bidRate0 == df.bidRate0.shift(-1)), (df.askRate0 == df.askRate0.shift(-1))
valsB, valsA = [0, (df.bidSize0 - df.bidSize0.shift(-1))], [0, (df.askSize0 - df.askSize0.shift(-1))]
defaultB, defaultA = df.bidSize0, df.askSize0

df['deltaVBid'] = np.select([b1,b2], valsB, default=defaultB)
df['deltaVAsk'] = np.select([a1,a2], valsA, default=defaultA)
df['VOI'] = df.deltaVBid - df.deltaVAsk

Wall time: 184 ms


In [48]:
df.tail()

,askRate0,askRate1,askRate2,askRate3,askRate4,askRate5,askRate6,askRate7,askRate8,askRate9,askRate10,askRate11,askRate12,askRate13,askRate14,askSize0,askSize1,askSize2,askSize3,askSize4,askSize5,askSize6,askSize7,askSize8,askSize9,askSize10,askSize11,askSize12,askSize13,askSize14,bidRate0,bidRate1,bidRate2,bidRate3,bidRate4,bidRate5,bidRate6,bidRate7,bidRate8,bidRate9,bidRate10,bidRate11,bidRate12,bidRate13,bidRate14,bidSize0,bidSize1,bidSize2,bidSize3,bidSize4,bidSize5,bidSize6,bidSize7,bidSize8,bidSize9,bidSize10,bidSize11,bidSize12,bidSize13,bidSize14,y,spread,midRate,expectedY,totalBidVol1,totalBidVol2,totalBidVol3,totalBidVol4,totalBidVol5,totalBidVol6,totalBidVol7,totalBidVol8,totalBidVol9,totalBidVol10,totalBidVol11,totalBidVol12,totalBidVol13,totalBidVol14,totalAskVol1,totalAskVol2,totalAskVol3,totalAskVol4,totalAskVol5,totalAskVol6,totalAskVol7,totalAskVol8,totalAskVol9,totalAskVol10,totalAskVol11,totalAskVol12,totalAskVol13,totalAskVol14,bidAskRatio1,bidAskRatio2,bidAskRatio3,bidAskRatio4,bidAskRatio5,bidAskRatio6,bidAskRatio7,bidAskRatio8,bidAskRatio9,bidAskRatio10,bidAskRatio11,bidAskRatio12,bidAskRatio13,bidAskRatio14,vwaBid,vwaAsk,vwaBidDMid,vwaAskDMid,deltaVBid,deltaVAsk
2999994,1584.0,1584.0,1584.0,1585.0,1586.0,1586.0,1586.0,1587.0,1588.0,1588.0,1588.0,1589.0,1590.0,1590.0,1591.0,8.0,11.0,8.0,17.0,8.0,9.0,7.0,8.0,20.0,9.0,6.0,2.0,43.0,12.0,1.0,1583.0,1582.0,1582.0,1582.0,1581.0,1580.0,1580.0,1580.0,1579.0,1578.0,1578.0,1578.0,1577.0,1576.0,1576.0,4.0,15.0,10.0,10.0,36.0,20.0,64.0,13.0,28.0,44.0,95.0,2.0,3.0,26.0,3.0,0.5,0.5,1583.0,0.0,19.0,29.0,39.0,75.0,95.0,159.0,172.0,200.0,244.0,339.0,341.0,344.0,370.0,373.0,19.0,27.0,44.0,52.0,61.0,68.0,76.0,96.0,105.0,111.0,113.0,156.0,168.0,169.0,1.000000,1.074219,0.88623,1.442383,1.557617,2.337891,2.263672,2.083984,2.324219,3.054688,3.017578,2.205078,2.203125,2.207031,24.861683,11.269092,1558.138317,-1571.730908,0.0,-1.0
2999995,1584.0,1584.0,1584.0,1585.0,1586.0,1586.0,1586.0,1587.0,1588.0,1588.0,1588.0,1589.0,1590.0,1590.0,1591.0,9.0,10.0,8.0,17.0,8.0,9.0,7.0,8.0,20.0,9.0,6.0,2.0,43.0,12.0,1.0,1583.0,1582.0,1582.0,1582.0,1581.0,1580.0,1580.0,1580.0,1579.0,1578.0,1578.0,1578.0,1577.0,1576.0,1576.0,4.0,15.0,10.0,10.0,36.0,20.0,64.0,13.0,28.0,44.0,95.0,2.0,3.0,26.0,3.0,0.5,0.5,1583.0,0.0,19.0,29.0,39.0,75.0,95.0,159.0,172.0,200.0,244.0,339.0,341.0,344.0,370.0,373.0,19.0,27.0,44.0,52.0,61.0,68.0,76.0,96.0,105.0,111.0,113.0,156.0,168.0,169.0,1.000000,1.074219,0.88623,1.442383,1.557617,2.337891,2.263672,2.083984,2.324219,3.054688,3.017578,2.205078,2.203125,2.207031,24.861683,11.269092,1558.138317,-1571.730908,0.0,-1.0
2999996,1584.0,1584.0,1584.0,1585.0,1586.0,1586.0,1586.0,1587.0,1588.0,1588.0,1588.0,1589.0,1590.0,1590.0,1591.0,10.0,9.0,8.0,17.0,8.0,9.0,7.0,8.0,20.0,9.0,6.0,2.0,43.0,12.0,1.0,1583.0,1582.0,1582.0,1582.0,1581.0,1580.0,1580.0,1580.0,1579.0,1578.0,1578.0,1578.0,1577.0,1576.0,1576.0,4.0,15.0,10.0,10.0,36.0,20.0,64.0,13.0,28.0,44.0,95.0,2.0,3.0,26.0,3.0,0.5,0.5,1583.0,0.0,19.0,29.0,39.0,75.0,95.0,159.0,172.0,200.0,244.0,339.0,341.0,344.0,370.0,373.0,19.0,27.0,44.0,52.0,61.0,68.0,76.0,96.0,105.0,111.0,113.0,156.0,168.0,169.0,1.000000,1.074219,0.88623,1.442383,1.557617,2.337891,2.263672,2.083984,2.324219,3.054688,3.017578,2.205078,2.203125,2.207031,24.861683,11.269092,1558.138317,-1571.730908,0.0,-1.0
2999997,1584.0,1584.0,1584.0,1585.0,1586.0,1586.0,1586.0,1587.0,1588.0,1588.0,1588.0,1589.0,1590.0,1590.0,1591.0,11.0,8.0,8.0,17.0,8.0,9.0,7.0,8.0,20.0,9.0,6.0,2.0,43.0,12.0,1.0,1583.0,1582.0,1582.0,1582.0,1581.0,1580.0,1580.0,1580.0,1579.0,1578.0,1578.0,1578.0,1577.0,1576.0,1576.0,4.0,15.0,10.0,10.0,36.0,20.0,64.0,13.0,28.0,44.0,95.0,2.0,3.0,26.0,3.0,0.5,0.5,1583.0,0.0,19.0,29.0,39.0,75.0,95.0,159.0,172.0,200.0,244.0,339.0,341.0,344.0,370.0,373.0,19.0,27.0,44.0,52.0,61.0,68.0,76.0,96.0,105.0,111.0,113.0,156.0,168.0,169.0,1.000000,1.074219,0.88623,1.442383,1.557617,2.337891,2.263672,2.083984,2.324219,3.054688,3.017578,2.205078,2.203125,2.207031,24.861683,11.269092,1

# Technical Analysis

# Feature Selection

# Cross-validation